In [1]:
%pip install -U sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3
import shutil
import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.221.1
sagemaker role arn: arn:aws:iam::236241703319:role/service-role/AmazonSageMaker-ExecutionRole-20240525T225318
sagemaker bucket: sagemaker-ap-northeast-2-236241703319
sagemaker session region: ap-northeast-2


In [3]:
model_data = 's3://sagemaker-ap-northeast-2-236241703319/neuron-experiments/bge-m3/compiled-model/compiled_model.tar.gz'

In [4]:
import logging
from sagemaker.utils import name_from_base
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor
from datetime import datetime

hardware = "inf1"
date_string = datetime.now().strftime("%Y%m-%d%H-%M%S")

pytorch_model = PyTorchModel(    
    # image_uri=f"763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuron:1.13.1-neuron-py310-sdk2.12.0-ubuntu20.04",
    model_data=model_data,
    role=role,
    name=name_from_base('bge-m3'),
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    framework_version="1.9.1",
    py_version="py38",
    entry_point="inference.py",
    model_server_workers=4, # 1 worker per core
    env = {
        'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '7200' 
    }
)
pytorch_model._is_compiled_model = True

In [6]:
%%time
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.inf1.6xlarge",
    endpoint_name=f"bge-m3-{hardware}-{date_string}",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

Defaulting to only supported image scope: training. Ignoring image scope: inference.
Defaulting to the only supported framework/algorithm version: 1.11.0. Ignoring framework/algorithm version: 1.9.1.


ValueError: Unsupported region: ap-northeast-2. You may need to upgrade your SDK version (pip install -U sagemaker) for newer regions. Supported region(s): ap-northeast-1, ap-south-1, ap-south-2, ap-southeast-1, ap-southeast-2, ap-southeast-4, eu-central-1, eu-central-2, eu-south-2, eu-west-1, eu-west-3, il-central-1, sa-east-1, us-east-1, us-east-2, us-west-2, ca-west-1.

In [25]:
sagemaker.__version__

'2.221.1'

In [27]:
region

'ap-northeast-2'